In [1]:
from src import Instance , Environment, Reporter
# from component.Ying import Enhancer
from component.FindMarking_v2 import FindMarking
from component.ZeroDCE import Enhancer
from component.DeepLab import Segmentation
from component.OrientNet import OrientationCorrection
from component.CRAFT import CharExtraction
from component.CharNet import CharRecognition
from component.MatchMarking import MatchMarking

import os
from pathlib import Path

# detect_dir = './Selected_data'


# Set Environment
env = Environment(1, test=True)


# Load Components
find_marking = FindMarking(env, mk_file='Location_Char.xls',)
enhancer = Enhancer(env,'./checkpoints_v2/Epoch99.pth')
# enhancer = Enhancer(env)
segment = Segmentation(env)
orient = OrientationCorrection(env, checkpoint='./checkpoints/googlenet_checkpoint_100.pth',num_classes=8)
# extract = CharExtraction(env, checkpoint='./checkpoints/CRAFT_clr_best.pth')
extract = CharExtraction(env, checkpoint='../spotting/weights_04_10_1/CRAFT_clr_best.pth')
# extract = CharExtraction(env, checkpoint='../spotting/weights_04_12_no_padding/CRAFT_clr_best.pth')
recognize = CharRecognition(env,  checkpoint='./checkpoints_v2/checkpoint_best_params.pth', dependencies = [extract], num_class=37 )
match = MatchMarking(env, dependencies=[find_marking, recognize])


/home/czs/anaconda3/lib/python3.8/site-packages/torchvision/models/googlenet.py:71: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


In [4]:
detect_dir = Path('/home/czs/_lab/PaperCompExp/test_data/e2e_enh_data')
files = detect_dir.glob('*.jpeg')

reporter = Reporter()
from PIL import Image
import cv2
import json
import time
import random

def pp(img):
    display(Image.fromarray(img))

eval_results = dict()
eval_results['time']=0
eval_results['data']=dict()
for idx, file in enumerate(files):
    
#     if not file.name.endswith('_1.jpg'):
#         continue
    
    ins = Instance(os.path.join(detect_dir, file))
    raw = ins.raw
    raw_h, raw_w = raw.shape[:2]
    
#     ins = find_marking(ins)
#     ins = enhancer(ins)
    ins = segment(ins)
#     ins = orient(ins)
#     rotater = cv2.getRotationMatrix2D((64,64), random.randint(0,3)*90, 1.0)
#     ins.img = cv2.warpAffine(ins.img, rotater, (128,128))
    
    ins = extract(ins)
    ins = recognize(ins)
    recognize.judge(ins,True)
#     ins = match(ins)
    print(idx,file.name,ins.text)
    eval_results['data'][file.stem+'_1.jpg'] = ins.text

#     img_p = ins.img.copy()
#     for l,t,r,b in ins.char_boxes:
#         cv2.rectangle(img_p,(l,t),(r,b),(0,0,255),2)

    
    reporter.record(ins)
    
#     reporter.reportLatest()
    if ins.result[0] == False:
        pp(ins.text_map)
        pp(ins.link_map)
        print(len(ins.char_boxes))
        pp(img_p)
        
Path('inf_result_2.json').write_text(json.dumps(eval_results))
# 打印时间报告
reporter.reportTime()
# 打印结果报告
reporter.reportInseption()
# reporter.exportHTML('reports/report_0105_8')

0 0703 FV993041HMSLF47AWJ70321-U2040@5_side.jpeg 2H
1 1056 FV993041JPJLF47AXJ70321-U8190@5_side.jpeg AZ
2 0793 FV992851RFWLF14AQJ70154-U5200@5_side.jpeg 2H
3 0753 FV993041HDDLF47AZJ70321-U4780@1_side.jpeg 5G
4 0432 FV993041HALLF47A1J70321-Q6950@2_side.jpeg DI915
5 0576 -Q8400@5_side.jpeg 638ZT
6 0422 FV992830VK9LF14A0J70135-Q6950@2_side_偏移.jpeg DI914
7 0592 FV992820PFSLF14A2J70131-Q8401@5_side_桥连.jpeg GBIKA1CFDMC86106Z
8 0385 FV993041GLWLF47AWJ70321-Q6862@3_side.jpeg UI9O4
9 0243 FV992741B1XLF3GAXJ70033-Q3431@2_side_偏移.jpeg 289VA
10 0395 FV992741B6XLF3GAGJ70033-Q6865@3_side_偏移.jpeg HHJ914
11 0779 FV993041HDJLF47AUJ70321-U4802@4_side.jpeg VP9SA
12 0916 FV993041G6BLF47APJ70321-U7210@1_side_偏移.jpeg S1CS32BELAT930
13 0919 FV992772VQ7LF14AEJ70098-U7230@5_side.jpeg 8DK8EZFDNF5812
14 0984 FV993041G0WLF47ANJ70321-U7430@1_side_偏移.jpeg S7C532BELAT930
15 0307 FV993041J0LLF47AWJ70321-Q3451@1_side.jpeg ANY
16 0066 FV992851TQQLF47APJ70166-D3300@3_side.jpeg O-95
17 0309 FV993041J2TLF47AHJ70321-Q3451@

In [3]:
reporter.exportHTML('reports/report_0105_9')

HTML report has been saved at reports/report_0105_9_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1.html
